In [1]:
%load_ext autoreload
%autoreload 2

from __future__ import division
import numpy as np
import scipy.signal as sg
from scipy.fftpack import rfft, fftfreq
from scipy.io import wavfile
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline
import matplotlib.colors as colors
from pydub import AudioSegment
import math as m
import tensorflow as tf
import utilities as util

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [5]:
guitar_note_fs,  guitar_note_wav,  guitar_note_freq,  guitar_note_time,  guitar_note_spec,  guitar_note_argsort  = util.gather_data('../samples/Guitar_C_Major_Note.wav')

In [7]:
nps_uniq, dfreqs, notes = util.precompute_spect()

In [44]:
def tf_normal_peak(vals, dfreq):
    if vals.shape[0] == 1: return vals
    sums = tf.reduce_sum(vals, 0, keepdims=True)
    weights = vals/sums
    freqs = tf.range(tf.cast(tf.shape(vals)[0], 'float64'), dtype='float64')
    mean = tf.reduce_sum(
        tf.cast(weights, 'float64') * tf.cast(freqs[:,None], 'float64'),  
        axis=0,
        keepdims=True)
    
    var = tf.truediv(tf.reduce_sum(tf.cast(((freqs[:,None]-mean)**2), 'float64')*tf.cast(weights, 'float64'), 0),dfreq)
    denom_var = tf.multiply(tf.constant(2.0, dtype='float64'), m.pi)
    denom_var_mult = tf.scalar_mul(denom_var, var)
    denom = tf.sqrt(tf.abs(denom_var_mult))
    dst_peak = tf.truediv(tf.constant(1.0, dtype='float64'),denom)
    peak = (tf.cast(dst_peak, 'float64') * tf.cast(dfreq, 'float64') * tf.cast(tf.squeeze(sums), 'float64')) if (dst_peak != 'Nan') else 0
    return peak

In [10]:
smallest_nps = 44100/24 * 4/3 
smallest_nps = 2**int(np.log2(smallest_nps))

In [45]:
full_spec = []
for nperseg in nps_uniq:
    print('...')
    signals = tf.placeholder(tf.float32)

    frames = tf.contrib.signal.frame(signals, frame_length=np.int32(nperseg),
                                     frame_step=np.int32(nperseg-smallest_nps))
    stfts = tf.contrib.signal.stft(signals, frame_length=np.int32(nperseg),
                                   frame_step=np.int32(nperseg-smallest_nps),
                                   fft_length=np.int32(nperseg))
#     power_spectrograms = tf.real(stfts * tf.conj(stfts))
    magnitude_spectrograms = tf.abs(stfts)
    full_spec.append(magnitude_spectrograms)

for nps_ind, slc in notes:
    tf_spec = full_spec[nps_ind]
    tf_np = tf_normal_peak(tf_spec[:,slc], dfreqs[nps_ind])


...
...
...
...
...
